In [1]:
import glob
import tifffile
import numpy as np
import lightgbm as lgb
import warnings
import tqdm

warnings.simplefilter('ignore')

In [5]:
train_path =  '/home/sebastien/Documents/projects/solafune-solar-panel-detection/data/train/s2_image/'
mask_path = '/home/sebastien/Documents/projects/solafune-solar-panel-detection/data/train/mask/'

masks = glob.glob(f'{mask_path}/*')
trains = glob.glob(f'{train_path}/*')
masks.sort()
trains.sort()

In [6]:
X = []
y = []
g = []

for i, (t, m) in enumerate(zip(trains, masks)):
    img = tifffile.imread(t).astype(np.float64)
    mask = tifffile.imread(m).astype(np.float64)
    X.append(img.reshape(-1,12))
    y.append(mask.reshape(-1))
    g.append(np.ones_like(mask.reshape(-1))*i)
    
X = np.vstack(X)
y = np.hstack(y)
g = np.hstack(g) 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from sklearn.svm import LinearSVC


gkfold = GroupKFold(n_splits=4)
models = []
for i, (train_idx, valid_idx) in enumerate(gkfold.split(X, y, g)):
    train_x = X[train_idx]
    train_y = y[train_idx]
    
    val_x = X[valid_idx]
    val_y = y[valid_idx]

    exported_pipeline = LinearSVC(C=0.01, dual=False, loss="squared_hinge", penalty="l1", tol=0.01)

    exported_pipeline.fit(train_x, train_y)
    
    # results = exported_pipeline.predict(X_test)
    models.append(results)

    

# X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, test_size=0.10)

In [ ]:
test_path =  '/home/sebastien/Documents/projects/solafune-solar-panel-detection/data/evaluation/'
test_mask_path = '/home/sebastien/Documents/projects/solafune-solar-panel-detection/data/sample/'
#test_path =  '/tmp/kaggledata/solafue_solars/train/s2_image/'
#test_mask_path = '/tmp/kaggledata/solafue_solars/train/mask/'

masks = glob.glob(f'{test_mask_path}/*')
tests = glob.glob(f'{test_path}/*')
masks.sort()
tests.sort()

In [ ]:
import os
if not os.path.isdir('output1'):
    os.mkdir('output1')

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from tpot.builtins import StackingEstimator
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, test_size=0.25)

# Average CV score on the training set was: 0.8222158445636321
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=RandomForestClassifier(bootstrap=True, criterion="gini", 
                                                       max_features=1.0, min_samples_leaf=4, min_samples_split=8, 
                                                       n_estimators=100)),
    RandomForestClassifier(bootstrap=False, criterion="entropy", max_features=0.25, 
                           min_samples_leaf=2, min_samples_split=5, n_estimators=100)
 )


exported_pipeline.fit(X_train, y_train)
results = exported_pipeline.predict(X_test)

In [ ]:
# import numpy as np
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.svm import LinearSVC

# X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, test_size=0.25)

# # Average CV score on the training set was: 0.7454340084081921
# exported_pipeline = LinearSVC(C=0.01, dual=False, loss="squared_hinge", penalty="l1", tol=0.01)

# exported_pipeline.fit(X_test, y_test)

# results = exported_pipeline.predict(X_test)

In [ ]:
from sklearn.metrics import f1_score
f1 = f1_score(y_test, results)

In [ ]:
f1

In [ ]:
import os
if not os.path.isdir('output2'):
    os.mkdir('output2')

In [ ]:
for i, (m, t) in tqdm.tqdm(enumerate(zip(masks, tests))):
    basename = os.path.basename(m)
    output_file = f'output1/{basename}'
    
    img = tifffile.imread(t).astype(np.float64)
    mask = tifffile.imread(m).astype(np.float64)
    
    X = img.reshape(-1, 12) 
    shape_mask = mask.shape
    print(shape_mask)
    
    pred = 0
    # for model in models:
        # pred = model.predict_proba(X) / len(models)
    pred = exported_pipeline.predict(X)
    pred = pred.astype(np.uint8)

    # pred_mask = np.argmax(pred, axis=1).astype(np.uint8)
    pred_mask = pred.reshape(shape_mask[0], shape_mask[1])

    tifffile.imwrite(output_file, pred_mask)

In [ ]:
pred_mask.shape

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(pred_mask)

In [ ]:
pred_mask.shape

In [ ]:
type(pred_mask)

In [ ]:
np.unique(pred_mask)